In [2]:
import pandas as pd
import csv

# Load the data into a pandas dataframe
df = pd.read_csv('comments_economie.csv')

# Show the first few rows of the dataframe
print(df.head())

   Unnamed: 0                            postId  \
0           0  2f26392804e911eb98f1646e69d991ea   
1           1  2f26392804e911eb98f1646e69d991ea   
2           2  2f26392804e911eb98f1646e69d991ea   
3           3  2f26392804e911eb98f1646e69d991ea   
4           4  2f26392804e911eb98f1646e69d991ea   

                                             comment  score     topic  
0  هذه السنة الكل يحمل المسؤولية لكورونا،ولله الح...     12  economie  
1  القوانيين التشريعية تعيقها التماسيح و اغلبية ا...      7  economie  
2  قانون الشغل مثقوب من قبل من كورونا ، المشغلين ...      8  economie  
3  فراغات هي ف بزاف د الميادين كاينة قبل من كورون...      9  economie  
4  ياودي راه الفراغ كاين من شحال هادي ݣاع ماديروا...      8  economie  


In [3]:
print(df['score'])

0        12
1         7
2         8
3         9
4         8
         ..
28071   -14
28072     6
28073    18
28074    -1
28075    -3
Name: score, Length: 28076, dtype: int64


In [6]:
pip install tensorflow


     ------------------------------------ 266.3/266.3 MB 402.6 kB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 890.7 kB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 1.2 MB/s eta 0:00:00
     -------------------------------------- 895.9/895.9 kB 1.3 MB/s eta 0:00:00
     ------------------------------------ 439.2/439.2 kB 298.3 kB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 952.3 kB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 1.5 MB/s eta 0:00:00
     -------------------------------------- 23.2/23.2 MB 978.4 kB/s eta 0:00:00
     -------------------------------------- 177.8/177.8 kB 1.2 MB/s eta 0:00:00
     --------------------------------------

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
import csv 
import json 

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)
          
csvFilePath = r'comments_economie.csv'
jsonFilePath = r'comments.json'

csv_to_json(csvFilePath, jsonFilePath)

# Load the existing dataset into a pandas dataframe
df = pd.read_csv("comments_economie.csv")

# Randomly sample a portion of the data
df = df.sample(frac=0.9, random_state=1)

# Preprocess the text data
comments = df['comment'].tolist()
scores = df['score'].tolist()

# Tokenize the comments
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(comments)
sequence = tokenizer.texts_to_sequences(comments)
padded = pad_sequences(sequence, maxlen=100)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded, scores, test_size=0.2, random_state=1)
y_train = np.array(y_train)
y_val = np.array(y_val)


# Create the deep neural network model
model = Sequential([
    Embedding(10000, 32),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=32)

# Load the scraped data into a pandas dataframe
scraped_data = pd.read_json("comments.json")

# Extract the text field from the scraped data
scraped_comments = scraped_data['comment'].tolist()

# Preprocess the scraped comments
scraped_sequence = tokenizer.texts_to_sequences(scraped_comments)
scraped_padded = pad_sequences(scraped_sequence, maxlen=100)

# Make predictions on the scraped comments
scraped_scores = model.predict(scraped_padded)
print(scraped_scores)

Epoch 1/5
632/632 [==============================] - 9s 12ms/step - loss: -11843662.0000 - accuracy: 0.0595 - val_loss: -78557640.0000 - val_accuracy: 0.0671
Epoch 2/5
632/632 [==============================] - 7s 12ms/step - loss: -864530304.0000 - accuracy: 0.0595 - val_loss: -2611486720.0000 - val_accuracy: 0.0671
Epoch 3/5
632/632 [==============================] - 8s 12ms/step - loss: -7958888448.0000 - accuracy: 0.0595 - val_loss: -16547055616.0000 - val_accuracy: 0.0671
Epoch 4/5
632/632 [==============================] - 10s 15ms/step - loss: -33302210560.0000 - accuracy: 0.0595 - val_loss: -56664424448.0000 - val_accuracy: 0.0671
Epoch 5/5
878/878 [==============================] - 2s 2ms/step
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier

# Load the existing dataset into a pandas dataframe
df = pd.read_csv("comments_economie.csv")

# Select a smaller subset of the data
subset = df.sample(frac=0.1)

# Preprocess the text data
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(subset['comment'])

# Train the model
model = DecisionTreeClassifier()
model.fit(features, subset['score'])

# Load the scraped data into a pandas dataframe
scraped_data = pd.read_json("comments.json")

# Extract the text field from the scraped data
scraped_comments = scraped_data['comment'].tolist()

# Preprocess the scraped comments
scraped_features = vectorizer.transform(scraped_comments)

# Make predictions on the scraped comments
scraped_scores = model.predict(scraped_features)
print(scraped_scores)

[12 39 41 ...  0 12  5]


In [9]:
# Evaluate the accuracy of the model
accuracy = model.score(features, subset['score'])
print("Accuracy: ", accuracy)

Accuracy:  0.9982193732193733


In [10]:
# Create a new dataframe from the scraped comments and their scores
scraped_df = pd.DataFrame({'comments': scraped_comments, 'scores': scraped_scores})

# Iterate over the rows of the dataframe and print the comment and its score
for index, row in scraped_df.iterrows():
    print("Comment: ", row['comments'])
    print("Score: ", row['scores'])
    print("\n")

Comment:  هذه السنة الكل يحمل المسؤولية لكورونا،ولله الحمد لقيتو السبة، أما السنوات الفارطة طفرتوه، الله يهديكم، الوباء فرش الجميع والشعب عاق بيكم لا أحزاب لا حكومة الكل عطيها لقصاير .
Score:  12


Comment:  القوانيين التشريعية تعيقها التماسيح و اغلبية البرلمانيين ارا نغطيو الشمس بالغربال
الان كلشي سبب هو كرونا
لاحول و لا قوة الا بالله العلي العظيم
Score:  39


Comment:  قانون الشغل مثقوب من قبل من كورونا ، المشغلين يستعبدون المغاربة بأبسط الأجور ، و الأغلبية الساحقة غير مصرح بيها كليا أو جزئيا للتهرب الضريبي . حنى مواطنين مكنتخلصوش و فراسنا هادشي ولكن واش هادشي فراس صحاب التقاعد الإستثنائي و 7 مليون في الصالير
Score:  41


Comment:  فراغات هي ف بزاف د الميادين كاينة قبل من كورونا !
دابا لي بغا يمسح شي فراغ ، إقولك : كورونا .
Score:  9


Comment:  ياودي راه الفراغ كاين من شحال هادي ݣاع ماديروا السبة بكورونا.
ز
Score:  3


Comment:  واحد خلص خدامة عندو حتآآآ أصبح وزيرا !!!!!
Score:  2


Comment:  الفراغ التشريعي الوطني في الشغل لم تأت به كورونا بل هو خلل كان موجودا واستفحل في عهد كورونا

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Comment:  بحكم قريب من الربابنة و عالم بما يدور في الخفاء، لا يسني الا ان اقول لاااااا حول و لا قوة إلا بالله. و الفاهم يفهم.
Score:  3


Comment:  السلام عليكم،
احسن ما يمكن للربابنة فعله هو التنازل عن ما لا يقل عن ربع أجرتهم الشهرية للمساهمة في التخفيف من اثار كورونا على الشركة الوطنية "لارام" ، اما المساهمة الرمزية فلن تفيد حتى لدفع اجور عمال الأمتعة او النظافة، 
الألمان لما يَرَوْن ان شركتهم لا تحقق ارباحا او مهددة بالإفلاس يتخذون قرار خفض اجورهم كمساهمة وهذه هي الوطنية الحقة ولا مكان للحلول الترقيعية، مجرد رأي ولكم واسع النظر، 
بالمناسبة ، يمكن لموظفي شركات الدولة والخواص اتخاذ نفس القرار تعبيرا عن وطنيتهم وبالخصوص أولائك الذين يتقاضون أجورا خيالية تفوق عشرات المرات الحد الأدنى للأجور،
Score:  5


Comment:  15 مليون شهريا + 45 مليون مكافاة كل سنة! وكأنكم تسوقون مكوكا فضائيا، لارام هي الاغلى من حيث الاسعار و الأسوأ من حيث الخدمات، نتمنى خوصصتها او تفكيكها وفتح المجال الجوي أمام المنافسة، تذكرة من باريس الدار البيضاء 2000 درهم في الوقت العادي مقابل 350 درهم عند الشركات الاوروبية!
Sc

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
print(scraped_df['scores'])

0        12
1        39
2        41
3         9
4         3
         ..
28071   -14
28072     1
28073     0
28074    12
28075     5
Name: scores, Length: 28076, dtype: int64


In [12]:
#Create a sentiment column
sentiments = []
for score in scraped_scores:
    if score > 0:
        sentiments.append("positive")
    elif score < 0:
        sentiments.append("negative")
    else:
        sentiments.append("neutral")

#Add the sentiment column to the scraped_data dataframe
scraped_data['sentiment'] = sentiments

In [13]:
print(scraped_data['sentiment'])

0        positive
1        positive
2        positive
3        positive
4        positive
           ...   
28071    negative
28072    positive
28073     neutral
28074    positive
28075    positive
Name: sentiment, Length: 28076, dtype: object


In [14]:
print(scraped_data)

                                        postId  \
0          0  2f26392804e911eb98f1646e69d991ea   
1          1  2f26392804e911eb98f1646e69d991ea   
2          2  2f26392804e911eb98f1646e69d991ea   
3          3  2f26392804e911eb98f1646e69d991ea   
4          4  2f26392804e911eb98f1646e69d991ea   
...      ...                               ...   
28071  28071  05ced54204ee11eba9a6646e69d991ea   
28072  28072  05ced54204ee11eba9a6646e69d991ea   
28073  28073  05ced54204ee11eba9a6646e69d991ea   
28074  28074  05ced54204ee11eba9a6646e69d991ea   
28075  28075  05ced54204ee11eba9a6646e69d991ea   

                                                 comment  score     topic  \
0      هذه السنة الكل يحمل المسؤولية لكورونا،ولله الح...     12  economie   
1      القوانيين التشريعية تعيقها التماسيح و اغلبية ا...      7  economie   
2      قانون الشغل مثقوب من قبل من كورونا ، المشغلين ...      8  economie   
3      فراغات هي ف بزاف د الميادين كاينة قبل من كورون...      9  economie   
4      ياودي ر

In [15]:
#Create a new column in the scraped_data dataframe with the sentiment labels
scraped_data['sentiment'] = ['positive' if score > 0 else 'negative' if score < 0 else 'neutral' for score in scraped_scores]

#Save the updated scraped_data dataframe to a json file
scraped_data.to_json("scraped_data_with_sentiment.json")

In [16]:
print(scraped_data)

                                        postId  \
0          0  2f26392804e911eb98f1646e69d991ea   
1          1  2f26392804e911eb98f1646e69d991ea   
2          2  2f26392804e911eb98f1646e69d991ea   
3          3  2f26392804e911eb98f1646e69d991ea   
4          4  2f26392804e911eb98f1646e69d991ea   
...      ...                               ...   
28071  28071  05ced54204ee11eba9a6646e69d991ea   
28072  28072  05ced54204ee11eba9a6646e69d991ea   
28073  28073  05ced54204ee11eba9a6646e69d991ea   
28074  28074  05ced54204ee11eba9a6646e69d991ea   
28075  28075  05ced54204ee11eba9a6646e69d991ea   

                                                 comment  score     topic  \
0      هذه السنة الكل يحمل المسؤولية لكورونا،ولله الح...     12  economie   
1      القوانيين التشريعية تعيقها التماسيح و اغلبية ا...      7  economie   
2      قانون الشغل مثقوب من قبل من كورونا ، المشغلين ...      8  economie   
3      فراغات هي ف بزاف د الميادين كاينة قبل من كورون...      9  economie   
4      ياودي ر

In [17]:
# Save the updated scraped_data to a json file
scraped_data.to_json("scraped_data_with_sentiment.json", orient='records', force_ascii=False)